# Лабораторная работа №5
## Выполнил студент группы БСТ2201 Идрисов Мухамет

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [14]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [15]:

def make_lps(podstr):
    m=len(podstr)
    lps=[0]*m
    i=1
    length=0
    while i<m:
        if podstr[i]==podstr[length]:
            length+=1
            lps[i]=length
            i+=1
        else:
            if length>0:
                length=lps[length-1]
            else:
                lps[i]=0
                i+=1
    return lps


def KMP(vvod, podstr):
    ans=[]
    n=len(vvod)
    m=len(podstr)
    lps=make_lps(podstr)
    i=0
    j=0
    while ((n-i)>=(m-j)):
        if vvod[i]==podstr[j]:
            i+=1
            j+=1
        if j==m:
            ans.append(i-m)
            j=lps[j-1]
        elif (i<n and podstr[j]!=vvod[i]):
            if j>0:
                j=lps[j-1]
            else:
                i+=1
    return ans

vvod=input()
podstr=input()
ans=[str(i) for i in KMP(vvod,podstr)]
strprint="Найдены совпадения, начинающиеся с индексов: " + ", ".join(ans)
print(strprint)



Найдены совпадения, начинающиеся с индексов: 0, 3, 6


#### Упрощенный алгоритм Бойера-Мура

In [16]:
CHARS = 256
def bad_chars(podstr, m):
    badChar = [-1]*CHARS
    for i in range(m):
        badChar[ord(podstr[i])] = i
    return badChar
 
 
def search(vvod, podstr):
    m = len(podstr)
    n = len(vvod)
    badChar = bad_chars(podstr, m)
    ans=[]
    s = 0
    while(s <= n-m):
        j = m-1
        while j >= 0 and podstr[j] == vvod[s+j]:
            j -= 1
        if j < 0:
            ans.append(s)
            s += (m-badChar[ord(vvod[s+m])] if s+m < n else 1)
        else:
            s += max(1, j-badChar[ord(vvod[s+j])])
    return ans

vvod=input()
podstr=input()
ans=[str(i) for i in KMP(vvod,podstr)]
strprint="Найдены совпадения, начинающиеся с индексов: " + ", ".join(ans)
print(strprint)

Найдены совпадения, начинающиеся с индексов: 0, 3, 6


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [17]:
from heapq import heappop, heappush
from itertools import count

def solve_puzzle(board):
    # Расстояние Манхэттена
    def manhattan_distance(state):
        distance = 0
        for index, value in enumerate(state):
            if value == 0:
                continue
            target_index = value - 1
            current_row, current_col = divmod(index, 4)
            target_row, target_col = divmod(target_index, 4)
            distance += abs(current_row - target_row) + abs(current_col - target_col)
        return distance

    def get_neighbors(state):
        index = state.index(0)
        row, col = divmod(index, 4)
        neighbors = []
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # вверх, вниз, влево, вправо
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < 4 and 0 <= new_col < 4:
                new_index = new_row * 4 + new_col
                new_state = list(state)
                new_state[index], new_state[new_index] = new_state[new_index], new_state[index]
                neighbors.append(tuple(new_state))
        return neighbors

    target = tuple(range(1, 16)) + (0,)
    # Проверка на решаемость
    inversions = sum(board[i] > board[j] for i in range(len(board)) for j in range(i+1, len(board)) if board[i] and board[j])
    if inversions % 2 == 1:
        return None

    # A* алгоритм поиска
    queue = []
    parent = {tuple(board): None} 
    heappush(queue, (manhattan_distance(board), 0, tuple(board)))
    seen = {tuple(board): 0}
    unique = count()
    while queue:
        _, cost, state = heappop(queue)
        if state == target:
            path = []
            while state is not None:
                path.append(state)
                state = parent[state]
            return path[::-1]
        for neighbor in get_neighbors(state):
            neighbor_cost = cost + 1 
            if neighbor not in seen or neighbor_cost < seen[neighbor]:
                seen[neighbor] = neighbor_cost
                parent[neighbor] = state
                heappush(queue, (neighbor_cost + manhattan_distance(neighbor), next(unique), neighbor))
    return None

# Пример использования
initial_state = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 13,  14, 15]
solution = solve_puzzle(initial_state)
if solution:
    for step in solution:
        print(step)
    print(f'Было найдено решение в {len(solution)} шага(-ов)')
else:
    print("No solution exists")


(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 13, 14, 15)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 0, 14, 15)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
Было найдено решение в 4 шага(-ов)


### Вывод